In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,scale
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

In [2]:
# Dates have to extracted
# data = pd.read_csv('train.csv', parse_dates=['Dates'])
data = pd.read_csv('./dataset/train.csv', parse_dates=['Dates'])
test = pd.read_csv('./dataset/test.csv', parse_dates=['Dates'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867873 entries, 0 to 867872
Data columns (total 10 columns):
Dates         867873 non-null datetime64[ns]
Category      867873 non-null object
Descript      867873 non-null object
DayOfWeek     867873 non-null object
PdDistrict    867873 non-null object
Resolution    867873 non-null object
Address       867873 non-null object
X             867873 non-null float64
Y             867873 non-null float64
Id            867873 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 66.2+ MB


In [4]:
# No null values in the data-frame
data.isnull().values.any()

False

In [5]:
# Dates
data_week_dict = {
    'Monday': 1,
    'Tuesday':2,
    'Wednesday':3,
    'Thursday':4,
    'Friday':5,
    'Saturday':6,
    'Sunday':7
}

data['Hour'] = data.Dates.dt.hour
data['Minutes'] = data.Dates.dt.minute
data['Year'] = data.Dates.dt.year
data['Month'] = data.Dates.dt.month
data['Day'] = data.Dates.dt.day
data['DayOfWeekNum'] = data['DayOfWeek'].replace(data_week_dict)

test['Hour'] = test.Dates.dt.hour
test['Minutes'] = test.Dates.dt.minute
test['Year'] = test.Dates.dt.year
test['Month'] = test.Dates.dt.month
test['Day'] = test.Dates.dt.day
test['DayOfWeekNum'] = test['DayOfWeek'].replace(data_week_dict)

In [6]:
data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Hour,Minutes,Year,Month,Day,DayOfWeekNum
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,17,40,2013,6,28,5
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2,46,2004,2,19,4
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,0,1,2007,11,14,3
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,18,30,2007,12,27,4
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,17,2,2012,9,9,7


In [7]:
labelencoder = LabelEncoder()

In [8]:
data['ResolutionNum'] = labelencoder.fit_transform(data['Resolution'])
data['PdDistrictNum'] = labelencoder.fit_transform(data['PdDistrict'])
data['CategoryNum'] = labelencoder.fit_transform(data['Category'])

test['ResolutionNum'] = labelencoder.fit_transform(test['Resolution'])
test['PdDistrictNum'] = labelencoder.fit_transform(test['PdDistrict'])

In [9]:
data = data[data.X < -121]
data = data[data.Y < 40]

test = test[test.X < -121]
test = test[test.Y < 40]

In [10]:
def getCapsAddress(i):
    s=''
    for j in i.split():
        if(j.isupper()):
            s=s+' '+j
    return s[1:]

data['newAddress'] = data.Address.apply(lambda a:getCapsAddress(a))
test['newAddress'] = test.Address.apply(lambda a:getCapsAddress(a))

In [11]:
data['newAddressNum'] = labelencoder.fit_transform(data.newAddress)
test['newAddressNum'] = labelencoder.fit_transform(test.newAddress)

In [12]:
data['Address_CrossRoad'] = data['Address'].str.contains('/')
test['Address_CrossRoad'] = test['Address'].str.contains('/')

topN_address_list = data['Address'].value_counts()
topN_address_list = topN_address_list[topN_address_list >=100]
topN_address_list = topN_address_list.index
print(topN_address_list)

data['Address_clean'] = data['Address']
test['Address_clean'] = test['Address']
data.loc[~data['Address'].isin(topN_address_list), 'Address_clean'] = 'Others'
test.loc[~test['Address'].isin(topN_address_list), 'Address_clean'] = 'Others'
print(data.shape)

crossload = data[data['Address_clean'].str.contains('/')]
crossroad_list = crossload['Address_clean'].unique()
print(len(crossroad_list))

Index(['800 Block of BRYANT ST', '800 Block of MARKET ST',
       '2000 Block of MISSION ST', '1000 Block of POTRERO AV',
       '900 Block of MARKET ST', '0 Block of TURK ST', '0 Block of 6TH ST',
       '300 Block of ELLIS ST', '400 Block of ELLIS ST',
       '16TH ST / MISSION ST',
       ...
       '700 Block of OAK ST', '2300 Block of GOLDEN GATE AV',
       '0 Block of LAGUNA ST', '600 Block of ALABAMA ST',
       '14TH ST / FOLSOM ST', '900 Block of GRANT AV',
       '1100 Block of ELLIS ST', '700 Block of GONZALEZ DR',
       '1800 Block of SUTTER ST', '800 Block of MISSOURI ST'],
      dtype='object', length=1616)
(867806, 23)
475


In [13]:
for address in crossroad_list:
    address_split = address.split('/')
    reverse_address = address_split[1].strip() + ' / ' + address_split[0].strip()
    data.loc[data['Address_clean'] == reverse_address, 'Address_clean'] = address
    test.loc[test['Address_clean'] == reverse_address, 'Address_clean'] = address
crossload = data[data['Address_clean'].str.contains('/')]
crossroad_list = crossload['Address_clean'].unique()
print(len(crossroad_list))

le = LabelEncoder()
data['Address_clean_encode'] = le.fit_transform(data['Address_clean'])
print(data.shape)

321
(867806, 24)


In [14]:
le = LabelEncoder()
test['Address_clean_encode'] = le.fit_transform(test['Address_clean'])

In [15]:
corr = data.corr()
print(corr['CategoryNum'].sort_values(ascending=False))

CategoryNum             1.000000
Address_clean_encode    0.070796
Address_CrossRoad       0.069896
ResolutionNum           0.039676
Hour                    0.023701
Id                      0.016881
DayOfWeekNum            0.000537
Day                     0.000354
Month                  -0.000360
Year                   -0.016795
newAddressNum          -0.019639
Minutes                -0.021945
X                      -0.030525
PdDistrictNum          -0.039477
Y                      -0.058963
Name: CategoryNum, dtype: float64


In [16]:
# data.drop('Dates', axis=1, inplace=True)
# data.drop('Descript', axis=1, inplace=True)
# data.drop('DayOfWeek', axis=1, inplace=True)
# data.drop('PdDistrict', axis=1, inplace=True)
# data.drop('Resolution', axis=1, inplace=True)
# data.drop('Address', axis=1, inplace=True)
# data.drop('Street', axis=1, inplace=True)
# data.drop('Category', axis=1, inplace=True)
# data.drop('Minutes', axis=1, inplace=True)
# data.drop('newAddress', axis=1, inplace=True)
# data.drop('Id',axis=1,inplace=True)

In [17]:
features=['X','Y','Hour','Minutes','Year','Month','Day','DayOfWeekNum', 'PdDistrictNum',
          'Address_CrossRoad', 'Address_clean_encode']

In [ ]:
# for i in data.CategoryNum.unique():
#     print(i,labelencoder.inverse_transform(data.CategoryNum.unique())[i])
#     data[data.CategoryNum==i].hist(bins=50, figsize=(20,15))
#     plt.show()
data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# Random seed has been set - As per the guidlines of the competition
train, test = train_test_split(data, test_size=0.3, random_state=3, shuffle=True)

In [ ]:
ytrain_ = train['CategoryNum']
Xtrain_ = train[features]
ytest_ = test['CategoryNum']
Xtest_ = test[features]

In [18]:
y_train = data['CategoryNum']
X_train = data[features]
# y_test = test['CategoryNum']
X_test = test[features]

In [19]:
X_test.shape

(10000, 11)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',max_iter=1000)
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_test)
log_loss(y_test,pred)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(Xtrain_,ytrain_)
print(clf.score(Xtest_,ytest_))
pred = clf.predict_proba(Xtest_)
print(log_loss(ytest_,pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_state = 42
max_depth = 7
min_weight_fraction_leaf = 1
n_estimators = 500
n_jobs = -1

model = RandomForestClassifier(
    max_depth=max_depth,
    n_estimators=n_estimators,
    n_jobs=n_jobs,
    min_weight_fraction_leaf=min_weight_fraction_leaf
)

# model.fit(Xtrain_,ytrain_)
# print(model.score(Xtest_,ytest_))
# pred = model.predict_proba(Xtest_)
# print(log_loss(ytest_,pred))

In [20]:
import xgboost as xgb

seed = 42
max_depth = 8
learning_rate = 0.2
min_child_weight = 1
n_estimators = 100

model = xgb.XGBClassifier(
    objective='multi:softprob', 
    seed=seed, 
    max_depth=max_depth,
    nthread=8,
    n_jobs=-1,
    min_child_weight=min_child_weight,
    learning_rate=learning_rate,
    n_estimators = n_estimators
)

In [21]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=8, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [ ]:
score = -1 * cross_val_score(model, Xtrain_, ytrain_, scoring='neg_log_loss', cv=3, n_jobs=-1)

In [ ]:
print("Score = {0:.5f}".format(score.mean()))

In [ ]:
score

In [22]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=8, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [23]:
import pickle

In [24]:
pickle.dump(model, open("xgboost_wo_res61118.p", "wb"))

In [25]:
model = pickle.load(open("xgboost_wo_res61118.p", "rb"))
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=nan, n_estimators=100,
       n_jobs=-1, nthread=8, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [26]:
predictions = model.predict_proba(X_test)

In [27]:
submission = pd.DataFrame(predictions)
submission.columns = sorted(data.Category.unique())
submission['Id'] = test['Id']
submission

,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS,Id
0,0.000329,0.047752,0.000047,0.000070,0.040039,0.001537,0.000220,0.012811,0.004066,0.000346,...,0.013391,0.001818,0.000124,0.018340,0.021246,0.021253,0.013600,0.023159,0.001789,349598
1,0.003397,0.188772,0.000048,0.000114,0.039263,0.012404,0.001618,0.024159,0.013238,0.000266,...,0.002395,0.011339,0.000745,0.032607,0.014400,0.058346,0.012497,0.071852,0.018567,766313
2,0.000270,0.023572,0.000012,0.000069,0.001072,0.001237,0.022449,0.017093,0.008580,0.000020,...,0.001169,0.007435,0.000025,0.022264,0.000197,0.020563,0.003579,0.041592,0.008221,169887
3,0.000105,0.048635,0.000005,0.000049,0.000301,0.001657,0.001987,0.015083,0.002748,0.000383,...,0.002447,0.001166,0.000035,0.012188,0.000160,0.045330,0.041319,0.012400,0.002496,594704
4,0.000533,0.065291,0.000020,0.000200,0.002027,0.004224,0.000618,0.014719,0.013272,0.000030,...,0.001202,0.004062,0.000033,0.029254,0.001236,0.039361,0.024970,0.081431,0.007559,47900
5,0.000123,0.088313,0.000063,0.000196,0.001547,0.001076,0.000738,0.088880,0.003073,0.000118,...,0.005479,0.001757,0.000045,0.013856,0.000163,0.034617,0.028124,0.020678,0.002115,339260
6,0.001095,0.071003,0.000050,0.000540,0.100239,0.001090,0.000173,0.008980,0.001234,0.000403,...,0.010912,0.004266,0.000194,0.023762,0.005662,0.111646,0.080640,0.010531,0.002854,169575
7,0.000417,0.034517,0.000071,0.000087,0.000389,0.001229,0.000069,0.005018,0.000431,0.000023,...,0.001650,0.001810,0.000021,0.016448,0.000115,0.027919,0.164215,0.004473,0.003360,681483
8,0.000770,0.063472,0.002197,0.000275,0.059901,0.002412,0.000143,0.008034,0.001078,0.005520,...,0.009910,0.003971,0.000349,0.057396,0.005587,0.039487,0.020580,0.011187,0.003405,276938
9,0.000407,0.055427,0.000111,0.000134,0.101556,0.000752,0.000348,0.005042,0.001165,0.000576,...,0.004308,0.004249,0.000404,0.035369,0.006630,0.089430,0.059661,0.006333,0.003166,146369


In [28]:
submission.to_csv('submission.csv', index=False)